In [1]:
import nltk
from nltk.tag import pos_tag
from nltk.corpus import twitter_samples
from nltk.stem.wordnet import WordNetLemmatizer
import re, string
from nltk import FreqDist
import random
import psycopg2
from textblob import TextBlob 
from nltk.tokenize import word_tokenize
import pandas as pd

In [2]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('twitter_samples')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\alexi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\alexi\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [3]:
text = twitter_samples.strings('tweets.20150430-223406.json')
# twitter_samples
# text

In [4]:
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(pos_tag(tweet_tokens[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


In [5]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(tweet_tokens[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


In [6]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

print(remove_noise(tweet_tokens[0], stop_words))

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [8]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [9]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)


freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [125]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [126]:
positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

In [127]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

Accuracy is: 0.997


In [128]:
## Save Classifier Model
import pickle
filename = "my_classifier.sav"
pickle.dump(classifier, open(filename, 'wb'))

In [129]:
p="datasucks"
conn = psycopg2.connect(user = "alexis",
                                  password = f"{p}",
                                  host = "127.0.0.1",
                                  port = "3306",
                                  database = "postgres")

In [130]:
cursor = conn.cursor()
tweet_list = []
followers = []
cursor.execute("select * from tweets")
tweets = cursor.fetchall()
if not tweets:
    print("empty")
for row in tweets:
    for col in row:
        if type(col) is dict:
            tweet_list.append(col['text'])
            followers.append(col['user']['followers_count'])
print(tweet_list)

['RT @BbyDmplng403: @LouiseLowrey There are so many hilarious comments in You Tube @PatriotsSoapbox chat on @_saltycrystal show tonight.', 'RT @DanRodimer: Democrat frontrunner Michael Bloomberg says he wants to make the whole country "behave" while dressed as Mary Poppins.\n\nWel…', 'The latest COACH! https://t.co/Sz1NOFbZmK #election2020 #misleadingtweets', 'THERE IS NO ONE TO VOTE FOR! \nCentrists demand Valium to get them through #Election2020', 'RT @K_JeanPierre: Calling all voters!! Are you ready for November?\n\n👉🏿 https://t.co/axp6U4rlB8 RT to spread the word. 🗳️ #Election2020 http…', '@ewarren Team Warren! #PresidentWarren #DemocraticPrimaries #Election2020 https://t.co/trsEqFgRLB', 'RT @JC_SevenSeven: @RushLimbaughEIB @BoSnerdley @larryelder @MichaelBerrySho @RamonRoblesJr @SenWarren @realDonaldTrump #Election2020 Enjoy…', '@JRubinBlogger As a man and feminist I am flabbergasted at this Tweet from you. This says so much about your own bi… https://t.co/f4Ebef99Tj', "RT @Ellars

In [131]:
our_tweet_tokens = []
for each in tweet_list:
    our_tweet_tokens.append(nltk.word_tokenize(each))

In [132]:
print(our_tweet_tokens[3])

['THERE', 'IS', 'NO', 'ONE', 'TO', 'VOTE', 'FOR', '!', 'Centrists', 'demand', 'Valium', 'to', 'get', 'them', 'through', '#', 'Election2020']


In [133]:
clean_tweet_tokens = []

for each in our_tweet_tokens:
    clean_tweet_tokens.append(remove_noise(each, stop_words))

In [134]:
our_words = get_all_words(clean_tweet_tokens)


freq_dist_pos = FreqDist(our_words)
print(freq_dist_pos.most_common(10))

[('rt', 1497), ('http', 961), ('election2020', 634), ('’', 378), ('poll', 347), ('christians', 304), ('good', 240), ('…', 225), ('vote', 216), ('would', 215)]


In [135]:
loaded_model = pickle.load(open("my_classifier.sav", 'rb'))

In [147]:
tweet = []
sentiment = []
for each in clean_tweet_tokens[:50]:
    tweet.append(each)
    sentiment.append(loaded_model.classify(dict([token, True] for token in each)))                            